In [6]:
from collections import defaultdict
import kevlar

In [11]:
def apply_ambig_thresh_post_hoc(filename, thresh=10):
    part_counts = defaultdict(int)
    with kevlar.open(filename, 'r') as fh:
        reader = kevlar.vcf.VCFReader(fh)
        for call in reader:
            if call.filterstr != 'PASS':
                continue
            partid = call.attribute('PART')
            part_counts[partid] += 1
    with kevlar.open(filename, 'r') as fh:
        for line in fh:
            if 'PASS' in line:
                partid = line.split(';PART=')[1].split(';')[0]
                if part_counts[partid] > thresh:
                    line = line.replace('\tPASS\t', '\tAmbiguousCall\t')
            yield line

In [12]:
for cov in ('10', '20', '30', '50'):
    print('DEBUG: cov', cov)
    infilename = 'kevlar-calls-{:s}x-nohomopoly.vcf.gz'.format(cov)
    outfilename = 'kevlar-calls-{:s}x-nohomopoly-ambig-thresh.vcf.gz'.format(cov)
    with kevlar.open(outfilename, 'w') as fh:
        for line in apply_ambig_thresh_post_hoc(infilename, thresh=10):
            print(line, end='', file=fh)

DEBUG: cov 10
DEBUG: cov 20
DEBUG: cov 30
DEBUG: cov 50
